# BERT

## Importing the necessory libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

C:\Users\PC\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Reading the data

In [ ]:
df = pd.read_csv('compiled_output.csv')  

# Step 2: Remove rows with empty or irrelevant content
df = df[df['Text'].str.strip().notna()]  # Remove rows with empty strings
df = df[df['Text'].str.strip() != '']  # Remove rows with only whitespace
df = df[~df['Text'].str.contains(r'\[.*\]')]  # Remove rows with non-textual content like '[2]'

In [ ]:

# Step 3: Map labels to integers for 3 classes (Adjust this based on your exact label names)
label_mapping = {
    'background_claim': 0,  # Example: background_claim -> 0
    'own_claim': 1,         # Example: own_claim -> 1
    'data': 2               # Example: data -> 2
}
df['Label'] = df['Label'].map(label_mapping)

# Step 4: Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'].tolist(), df['Label'].tolist(), test_size=0.3, random_state=42
)

# Step 5: Tokenize the texts using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

# Convert to Dataset format
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Step 6: Load pre-trained BERT model for sequence classification (with 3 labels)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Step 7: Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluate after each epoch
    save_strategy="epoch",           # Save model after each epoch
)

# Step 8: Compute metrics function for multi-class classification
def compute_metrics(p):
    predictions = p.predictions.argmax(axis=-1)  # Convert logits to class predictions
    accuracy = accuracy_score(p.label_ids, predictions)
    return {'accuracy': accuracy}

# Step 9: Trainer setup
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # pass the compute metrics function
)

# Step 10: Train the model
trainer.train()

# Step 11: Evaluate the model
eval_results = trainer.evaluate()

print("\nEvaluation Results:")
print(eval_results)

# Step 12: Make predictions on the test set
test_preds = trainer.predict(test_dataset)
test_preds_labels = test_preds.predictions.argmax(axis=-1)

# Step 13: Accuracy and Classification Report
accuracy = accuracy_score(test_labels, test_preds_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(test_labels, test_preds_labels))


C:\Users\PC\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Program Files\Python312\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 10/1178 [02:37<4:58:53, 15.35s/it]

{'loss': 1.1166, 'grad_norm': 6.849336624145508, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/1178 [05:03<4:38:02, 14.41s/it]

{'loss': 1.1446, 'grad_norm': 9.413037300109863, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


  3%|▎         | 30/1178 [07:30<4:43:25, 14.81s/it]

...

{'train_runtime': 10167.3228, 'train_samples_per_second': 0.926, 'train_steps_per_second': 0.116, 'train_loss': 0.7029295143733, 'epoch': 1.0}


100%|██████████| 253/253 [22:50<00:00,  5.42s/it]



Evaluation Results:
{'eval_loss': 0.6162757873535156, 'eval_accuracy': 0.7389150359177608, 'eval_runtime': 1377.2063, 'eval_samples_per_second': 2.931, 'eval_steps_per_second': 0.184, 'epoch': 1.0}


100%|██████████| 253/253 [16:00<00:00,  3.80s/it]

Accuracy: 73.89%

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.55      0.62      1039
           1       0.70      0.85      0.77      1733
           2       0.85      0.74      0.79      1265

    accuracy                           0.74      4037
   macro avg       0.75      0.71      0.72      4037
weighted avg       0.75      0.74      0.74      4037

